# 1. PyTorch 中的模型保存

`torch.save(obj, f, pickle_module, pickle_protocol=2, _use_new_zipfile_serialization=False)`

**主要参数**：
- obj：保存的对象，可以是整个模型，也可以是 dict（以字典形式存储的各个网络层的参数，以字典形式存储的优化器的参数等）
- f：输出路径

<br/>

**保存模型有两种方式**

**保存整个 Module：**

这种方法比较耗时，保存的文件大，**不推荐**。

`torch.save(net, path)`

**只保存模型的参数：**

运行比较快，保存的文件比较小，**推荐**。
```
state_dict = net.state_dict()
torch.save(state_sict, path)
```

<br/>

# 2. PyTorch 中的模型加载

`torch.load(f, map_location=None, pickle_module, **pickle_load_args)`

**主要参数：**
- f：文件路径
- map_location：指定存在 CPU 或者 GPU

<br/>

**加载模型也有两种方式**

**加载整个 Module：**

如果保存的是整个模型，那么加载时就加载整个模型。这种方法不需要事先创建一个模型对象，也不用知道模型的结构。

```
path_model = "./model.pkl"
net = torch.load(path_model)
```

**只加载模型的参数：**

如果保存的是模型的参数，那么加载时就加载参数。这种方法需要事先创建一个模型对象，再使用模型的 `load_state_dict()` 方法把参数加载到模型中。

```
from model import LeNet
net = LeNet(classes=2)

path_state_dict = "./model_state_dict.pkl"
state_dict_load = torch.load(path_state_dict)  # 将参数加载到内存中
net.load_state_dict(state_dict_load)  # 将参数加载到模型中，原有参数会被覆盖掉
```

<br/>

# 3. 模型的断点续训练

断点续训练是在训练过程中每隔一定数量的 epoch 就保存模型的参数、优化器参数和 epoch 等，如果训练意外终止了，就重新加载最后保存的的模型参数、优化器参数和 epoch 等，接着这个 checkpoint 继续训练。

**保存 checkpoint 的代码：**
```
checkpoint_interval = 5

# 每隔 5 个 epoch 就保存一个 checkpoint
if (epoch+1) % checkpoint_interval == 0:
    checkpoint = {"model_state_dict": net.state_dict(), "optimizer_state_dict": optimizer.state_dict(), "epoch": epoch}
    path_checkpoint = "./checkpoint_{}_epoch.pkl".format(epoch)
    torch.save(checkpoint, path_checkpoint)
```

**断点恢复训练的代码：**
```
# 恢复
path_checkpoint = "···"
checkpoint = torch.load(path_checkpoint)  # 加载到内存中

net.load_state_dict(checkpoint['model_state_dict'])  # 模型加载模型参数
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # 优化器加载优化器参数
start_epoch = checkpoint['epoch']  # 加载 epoch 数值

scheduler.last_epoch = start_epoch  # 设置 scheduler.last_epoch 为保存的 epoch

# 开始训练
for epoch in range(start_epoch + 1, MAX_EPOCH):  # 模型训练的起始 epoch 修改为保存的 epoch + 1
    ···